In [1]:
from pywinauto.application import Application
from types import SimpleNamespace
from pywinauto import clipboard
from PIL import ImageGrab

import pytesseract
import time

In [2]:
identity = SimpleNamespace()
identity.account = '39954163'
identity.password = '849152'

In [6]:
app = Application(backend='uia').start('C:\\国金证券同花顺独立下单\\xiadan.exe')

In [7]:
login = app.window(title="用户登录", class_name="#32770")
login.wait('visible')

<uiawrapper.UIAWrapper - '用户登录', Dialog, 3713040050290465231>

In [8]:
# set password
ipfd = login.child_window(auto_id="1012", class_name="Edit")
ipfd.set_text(identity.password)

# set verification codes
while True:
    time.sleep(5)
    ver_img = login.child_window(auto_id="1499", class_name="Static")
    box = (ver_img.rectangle().left, ver_img.rectangle().top, ver_img.rectangle().right-14, ver_img.rectangle().bottom)
    im = ImageGrab.grab().crop(box)
    verification_code = pytesseract.image_to_string(im)
    print('verification codes:', verification_code)
    ipfd = login.child_window(auto_id="1003", class_name="Edit")
    ipfd.set_text(verification_code)

    time.sleep(2)
    button_ok = login.child_window(auto_id="1006", class_name="Button")
    button_ok.click()

    try:
        dlg = app.window(title="网上股票交易系统5.0")
        dlg.wait('visible', timeout=5)
    except Exception as e:
        login.child_window(title="确定", auto_id='2').click()
        continue
    
    try:
        button_ok = dlg.child_window(title="确定", auto_id='1')
        button_ok.click()
    except Exception as e:
        pass

    break

verification codes: RBY2


In [10]:
tab = dlg.child_window(auto_id="1001", class_name="CCustomTabCtrl")
click(coords=(tab.rectangle().left+120, tab.rectangle().top+10))

In [11]:
ipfd = dlg.child_window(auto_id="1012", class_name="Edit")
ipfd.wait('visible')
ipfd.set_text(identity.password)
dlg.child_window(auto_id="1", class_name="Button").click()

<uia_controls.ButtonWrapper - '确定', Button, 3713040262629909031>

In [17]:
query = dlg.child_window(title='查　　询', control_type="TreeItem")

In [19]:
click(coords=(query.rectangle().left+10, query.rectangle().top+10))

In [20]:
query = dlg.child_window(title='查询资产', control_type="TreeItem")
click(coords=(query.rectangle().left+10, query.rectangle().top+10))

301.59

62689.0

In [ ]:
# 当前 余额
el = dlg.child_window(auto_id="10000", class_name="Static")
remain = float(el.window_text())
remain

In [50]:
# 证券市值
el = dlg.child_window(auto_id="10006", class_name="Static")
stocks_value = float(el.window_text())
stocks_value

In [73]:
# 获取 表格 内容
el = dlg.child_window(auto_id="1047", class_name="CVirtualGridCtrl")
el.type_keys('^A^C')
ver_img = dlg.child_window(auto_id="2405", class_name="Static")
box = (ver_img.rectangle().left, ver_img.rectangle().top, ver_img.rectangle().right+3, ver_img.rectangle().bottom)
im = ImageGrab.grab().crop(box)
verification_code = pytesseract.image_to_string(im)
print('verification codes:', verification_code)
ipfd = dlg.child_window(auto_id="2404", class_name="Edit")
ipfd.set_text(verification_code)
dlg.child_window(auto_id="1", class_name="Button").click()
clipboard.GetData()

verification codes: 3201


'\r\nfrom pywinauto import clipboard'

In [75]:
text = clipboard.GetData()

In [86]:
keys = []
stocks = []
for row_idx, line in enumerate(text.split('\r\n')):
    stock = {}
    for col_idx, value in enumerate(line.split('\t')):
        if row_idx == 0 and value.strip():
            keys.append(value.strip())
        elif row_idx !=0 and value.strip():
            if col_idx >= len(keys):
                break
            stock[keys[col_idx]] = value.strip()
    if stock:
        stocks.append(stock)
stocks

[{'证券代码': '600869',
  '证券名称': '智慧能源',
  '股票余额': '13900',
  '可用余额': '13900',
  '冻结数量': '0',
  '参考成本价': '4.719',
  '市价': '4.510',
  '市值': '62689.000',
  '参考盈亏': '-4372.730',
  '参考盈亏比例': '-4.429',
  '交易市场': '上海Ａ股',
  '股东帐户': 'E032622618',
  '证券集中度': '99.52%'}]